In [1]:
import pandas as pd
from icecream import ic
import joblib 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
january_df = pd.read_parquet('./yellow_tripdata_2023-01.parquet')
febreuary_df = pd.read_parquet('./yellow_tripdata_2023-02.parquet')

In [3]:
#Q1
ic('Q1 Number of columns for January:', january_df.shape[1])

ic| 'Q1 Number of columns for January:', january_df.shape[1]: 19


('Q1 Number of columns for January:', 19)

In [4]:
january_df['duration'] = january_df.tpep_dropoff_datetime - january_df.tpep_pickup_datetime
january_df.duration = january_df.duration.apply(lambda td: td.total_seconds() / 60)

In [5]:
febreuary_df['duration'] = febreuary_df.tpep_dropoff_datetime - febreuary_df.tpep_pickup_datetime
febreuary_df.duration = febreuary_df.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
#Q2
ic('Q2 Standard deviation for duration of January:', round(january_df.duration.std(),2))

ic| 'Q2 Standard deviation for duration of January:': 'Q2 Standard deviation for duration of January:'
    round(january_df.duration.std(),2): 42.59


('Q2 Standard deviation for duration of January:', 42.59)

In [7]:
febreuary_df.rename(columns={'Airport_fee': 'airport_fee'}, inplace=True)

In [8]:
# total_df = pd.concat([january_df, febreuary_df])
# total_df.head()

In [9]:

total_df_no_outliers = january_df[(january_df.duration >= 1) & (january_df.duration <= 60)].copy()

In [10]:
#Q3
ic('Q3 Percetage of remaining rows: ', 100*round(total_df_no_outliers.shape[0]/january_df.shape[0],2)) 

ic| 'Q3 Percetage of remaining rows: ': 'Q3 Percetage of remaining rows: '
    100*round(total_df_no_outliers.shape[0]/january_df.shape[0],2): 98.0


('Q3 Percetage of remaining rows: ', 98.0)

In [11]:
categorical = ['PULocationID', 'DOLocationID']
total_df_no_outliers[categorical] = total_df_no_outliers[categorical].astype(str)

In [12]:
#apply one hot encoding for PULocationID and DOLocationID
total_df_no_outliers_reduced = pd.get_dummies(total_df_no_outliers[categorical], columns=categorical).copy()
print(total_df_no_outliers.shape)
total_df_no_outliers_reduced.head()


(3009173, 20)


,PULocationID_1,PULocationID_10,PULocationID_100,PULocationID_101,PULocationID_102,PULocationID_106,PULocationID_107,PULocationID_108,PULocationID_109,PULocationID_11,...,DOLocationID_90,DOLocationID_91,DOLocationID_92,DOLocationID_93,DOLocationID_94,DOLocationID_95,DOLocationID_96,DOLocationID_97,DOLocationID_98,DOLocationID_99
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
total_df_no_outliers_reduced_list = total_df_no_outliers_reduced.to_dict(orient='records')
total_df_no_outliers_reduced_list[:2]

[{'PULocationID_1': False,
  'PULocationID_10': False,
  'PULocationID_100': False,
  'PULocationID_101': False,
  'PULocationID_102': False,
  'PULocationID_106': False,
  'PULocationID_107': False,
  'PULocationID_108': False,
  'PULocationID_109': False,
  'PULocationID_11': False,
  'PULocationID_111': False,
  'PULocationID_112': False,
  'PULocationID_113': False,
  'PULocationID_114': False,
  'PULocationID_115': False,
  'PULocationID_116': False,
  'PULocationID_117': False,
  'PULocationID_118': False,
  'PULocationID_119': False,
  'PULocationID_12': False,
  'PULocationID_120': False,
  'PULocationID_121': False,
  'PULocationID_122': False,
  'PULocationID_123': False,
  'PULocationID_124': False,
  'PULocationID_125': False,
  'PULocationID_126': False,
  'PULocationID_127': False,
  'PULocationID_128': False,
  'PULocationID_129': False,
  'PULocationID_13': False,
  'PULocationID_130': False,
  'PULocationID_131': False,
  'PULocationID_132': False,
  'PULocationID_133'

In [14]:
dv = DictVectorizer()
X_train = dv.fit_transform(total_df_no_outliers_reduced_list)

In [15]:
#save the model to disk
joblib.dump(dv, 'dv.joblib')

['dv.joblib']

In [19]:
#print dimnesions of X_train
ic(X_train.shape)

ic| X_train.shape: (3009173, 515)


(3009173, 515)

In [16]:
# Now let's use the feature matrix from the previous step to train a model.
# Train a plain linear regression model with default parameters

target = 'duration'
y_train = total_df_no_outliers[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
# save the model to disk
joblib.dump(lr, 'model.joblib')

['model.joblib']

In [18]:
y_pred = lr.predict(X_train)

#Q5
ic('Q5 RMSE on train:', round(mean_squared_error(y_train, y_pred, squared=False),2))

/Users/darwincubi/Documents/Projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
ic| 'Q5 RMSE on train:': 'Q5 RMSE on train:'
    round(mean_squared_error(y_train, y_pred, squared=False),2): 7.65


('Q5 RMSE on train:', 7.65)

In [22]:
import gc

# Forzar la recolección de basura
gc.collect()

0

In [32]:
del total_df_no_outliers
del total_df_no_outliers_reduced
del total_df_no_outliers_reduced_list

In [24]:
febreuary_df_no_outliers = febreuary_df[(febreuary_df.duration >= 1) & (febreuary_df.duration <= 60)].copy()

In [28]:
febreuary_df_no_outliers[categorical] = febreuary_df_no_outliers[categorical].astype(str)

In [29]:
febreuary_df_no_outliers_reduced = pd.get_dummies(febreuary_df_no_outliers[categorical], columns=categorical).copy()
print(febreuary_df_no_outliers_reduced.shape)
febreuary_df_no_outliers_reduced.head()

(2855951, 514)


,PULocationID_1,PULocationID_10,PULocationID_100,PULocationID_101,PULocationID_102,PULocationID_105,PULocationID_106,PULocationID_107,PULocationID_108,PULocationID_109,...,DOLocationID_90,DOLocationID_91,DOLocationID_92,DOLocationID_93,DOLocationID_94,DOLocationID_95,DOLocationID_96,DOLocationID_97,DOLocationID_98,DOLocationID_99
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
febreuary_df_no_outliers_reduced_list = febreuary_df_no_outliers_reduced.to_dict(orient='records')
febreuary_df_no_outliers_reduced_list[:2]

[{'PULocationID_1': False,
  'PULocationID_10': False,
  'PULocationID_100': False,
  'PULocationID_101': False,
  'PULocationID_102': False,
  'PULocationID_105': False,
  'PULocationID_106': False,
  'PULocationID_107': False,
  'PULocationID_108': False,
  'PULocationID_109': False,
  'PULocationID_11': False,
  'PULocationID_112': False,
  'PULocationID_113': False,
  'PULocationID_114': False,
  'PULocationID_115': False,
  'PULocationID_116': False,
  'PULocationID_117': False,
  'PULocationID_118': False,
  'PULocationID_119': False,
  'PULocationID_12': False,
  'PULocationID_120': False,
  'PULocationID_121': False,
  'PULocationID_122': False,
  'PULocationID_123': False,
  'PULocationID_124': False,
  'PULocationID_125': False,
  'PULocationID_126': False,
  'PULocationID_127': False,
  'PULocationID_128': False,
  'PULocationID_129': False,
  'PULocationID_13': False,
  'PULocationID_130': False,
  'PULocationID_131': False,
  'PULocationID_132': False,
  'PULocationID_133'

In [31]:
X_val = dv.transform(febreuary_df_no_outliers_reduced_list)

In [33]:
y_val = febreuary_df_no_outliers[target].values

In [34]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/Users/darwincubi/Documents/Projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811817957524739